<a href="https://colab.research.google.com/github/Otsemeuno/Customer-Subscription/blob/main/Customer_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Customer Subscription
$(Adapted from Kaggle)$


## Project Statement
This data is about a subscription-based digital product offering for financial advisory that includes newsletters, webinars, and investment recommendations. The offering has a couple of varieties, annual subscription, and digital subscription. The product also provides daytime support for customers to reach out to a care team that can help them with any product-related questions and signup/cancellation-related queries.

The data set contains the following information:

<ul><li>customer sign-up and cancellation dates at the product level
<li>call center activity
<li>customer demographics
<li>product pricing info<ul>

## Objectives

Forecast 2022 year revenue if the company expects to add the same number of new customers as the previous year

In 2022, one of the company's key initiatives is to improve the customer experience, one of the capabilities is superior customer care service, and they are thinking about enhancing in-product support better but that takes beyond 2022 to happen. It is very important to forecast call center case volume accurately so that they can make appropriate staffing decisions. Need to predict daily call volumes and make recommendations on how to staff on an hourly basis.

In [ ]:
#importing drive
from google.colab import drive

In [ ]:
#importing all necessary libraries
import pandas as pd
import numpy as np

##Data Preparation

####Load the dataset

In [ ]:
# Mount Google Drive on Kernel
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
customer_cases = pd.read_csv('/content/drive/MyDrive/Kaggle project/customer_cases.csv', index_col =[0])

In [ ]:
customer_info = pd.read_csv('/content/drive/MyDrive/Kaggle project/customer_info.csv', index_col =[0])

In [ ]:
customer_product = pd.read_csv('/content/drive/MyDrive/Kaggle project/customer_product.csv', index_col =[0])

In [ ]:
product_info = pd.read_csv('/content/drive/MyDrive/Kaggle project/product_info.csv', index_col =[0])

In [ ]:
#display top 5 rows in the dataset
customer_cases.head()

,case_id,date_time,customer_id,channel,reason
1,CC101,2017-01-01 10:32:03,C2448,phone,signup
2,CC102,2017-01-01 11:35:47,C2449,phone,signup
3,CC103,2017-01-01 11:37:09,C2450,phone,signup
4,CC104,2017-01-01 13:28:14,C2451,phone,signup
5,CC105,2017-01-01 13:52:22,C2452,phone,signup


In [ ]:
#display top 5 rows in the dataset
customer_info.head()

,customer_id,age,gender
1,C2448,76,female
2,C2449,61,male
3,C2450,58,female
4,C2451,62,female
5,C2452,71,male


In [100]:
#display top 5 rows in the dataset
customer_product.head() 

,customer_id,product,signup_date_time,cancel_date_time,customer,Customer
1,C2448,prd_1,2017-01-01 10:35:09,NaN,1,1
2,C2449,prd_1,2017-01-01 11:39:29,2021-09-05 10:00:02,0,0
3,C2450,prd_1,2017-01-01 11:42:00,2019-01-13 16:24:55,0,0
4,C2451,prd_2,2017-01-01 13:32:08,NaN,1,1
5,C2452,prd_1,2017-01-01 13:57:30,2021-06-28 18:06:01,0,0


In [ ]:
#display top 5 rows in the dataset
product_info.head()

,name,price,billing_cycle
product_id,,,
prd_1,annual_subscription,1200,12
prd_2,monthly_subscription,125,1


In [ ]:
customer_cases.shape

(330512, 5)

In [ ]:
customer_info.shape

(508932, 3)

In [ ]:
customer_product.shape

(508932, 4)

In [ ]:
product_info.shape

(2, 3)

###Missing Variables

In [ ]:
customer_cases.isna().sum()

case_id        0
date_time      0
customer_id    0
channel        0
reason         0
dtype: int64

In [ ]:
customer_info.isna().sum()

customer_id    0
age            0
gender         0
dtype: int64

In [ ]:
customer_product.isna().sum()

customer_id              0
product                  0
signup_date_time         0
cancel_date_time    396447
dtype: int64

In [ ]:
product_info.isna().sum()

name             0
price            0
billing_cycle    0
dtype: int64

##Feature Engineering

In [99]:
customer_product['customer'] = customer_product.cancel_date_time.apply(lambda x: 1 if x is np.nan else 0)

In [74]:
customer_product.columns

Index(['customer_id', 'product', 'signup_date_time', 'cancel_date_time',
       'customer', 'Customer'],
      dtype='object')

In [ ]:
testset['time_category'] = testset.time.apply(lambda time: 1 
                                    if 5 <= time.hour <= 12 
                                    else 2 if 12 <= time.hour <= 17 
                                    else 3 if 17 <= time.hour <= 21 
                                    else 4)